<a href="https://colab.research.google.com/github/NeagrTavakol/Final_dataminning/blob/main/Copy_of_Final_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import libraries
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

In [3]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Read dev dataset**

In [3]:
dev_csv="/content/drive/MyDrive/Colab Notebooks/dev.csv"
dev_df = pd.read_csv(dev_csv,sep='\t')
dev_df=dev_df.drop("Unnamed: 0",axis=1)
dev_columns=dev_df.columns

**Read train dataset**

In [4]:
train_csv="/content/drive/MyDrive/Colab Notebooks/train.csv"
train_df = pd.read_csv(train_csv,sep='\t')
train_df=train_df.drop("Unnamed: 0",axis=1)
train_columns = train_df.columns

# **Tokenize Data**

**install packages**

In [70]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

     |████████████████████████████████| 432 kB 5.2 MB/s 
     |████████████████████████████████| 174 kB 31.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=6d5c6c0a624737ebfe540d7a55f32c3bbe8cab15cecc1390d2db46787eca7c26
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [71]:
# Import the package
import stanza

In [72]:
# Download an Persiabn model into the default directory
print("Downloading Persian model...")
stanza.download('fa')

2022-01-28 16:48:11 INFO: Downloading default packages for language: fa (Persian)...


2022-01-28 16:48:19 INFO: Finished downloading models and saved to /root/stanza_resources.


**create models**

In [73]:
# Build a Persian pipeline, with all processors by default
print("Building a Persian pipeline...")
fa_nlp = stanza.Pipeline('fa')

2022-01-28 16:48:22 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-01-28 16:48:22 INFO: Use device: gpu
2022-01-28 16:48:22 INFO: Loading: tokenize


Building a Persian pipeline...


2022-01-28 16:48:35 INFO: Loading: mwt
2022-01-28 16:48:35 INFO: Loading: pos
2022-01-28 16:48:35 INFO: Loading: lemma
2022-01-28 16:48:35 INFO: Loading: depparse
2022-01-28 16:48:36 INFO: Done loading processors!


**find tokens of dataframe and save in array**

In [ ]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
tokenized_array=[]
for i in range(0,len(train_df)[:52465]):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

In [ ]:
#find tokens of rest of dataset (process interupted cause of volume of dataset)
# rest_train_df=train_df[52466:]
for i in range(52466,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


**save tokens array to csv file**

In [ ]:
#save tokens to array
import csv
with open('tokens_array.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(tokenized_array)

**convert tokens array to dataframe **

In [ ]:
# convert tokenized_array to dataframe
tokenized_df=pd.DataFrame(tokenized_array)

**save dataframe to csv file**

In [ ]:
#save dataframe 
tokenized_df.to_csv('tokens_df.csv')

In [ ]:
# with open('tokens_array.csv', 'r', newline='') as file:
#   myreader = csv.reader(file, delimiter=',')
#   for rows in myreader:
#     print(rows)

# **Read tokenized dataet**

**read from array csv file **

In [ ]:
############

In [5]:
#read array csv file
tokens_array=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_array.csv",sep='\t',header=None)
# tokens_array.head()

# **Preproccessing**

**Remove punctuation**

In [6]:
from types import new_class
# from string import punctuation
import re

punctuation=['!','"','#','$','%','&',"'",'(',')','*','+','،','-','.',','"/",':',';','<','=','>','?','@','[',"\\","]","^",'_','`','{','|','}','~']
# tokens_array.to_numpy()
tokens_arr=[]
for i in tokens_array.to_numpy():
  sent=[]
  for j in i[0].split(','):
    if j not in punctuation:
      sent.append(j)
  tokens_arr.append(sent)

**Analyze Reviews Length**

In [152]:
# import matplotlib.pyplot as plt
reviews_len = [len(x) for x in tokens_arr]
reviews_len=np.array(reviews_len)

In [ ]:
# pd.Series(tokens_arr).hist()
# plt.show()
# pd.Series(tokens_arr).describe()

In [151]:
print("min:",reviews_len.min(),"max:",reviews_len.max(),"min:",reviews_len.mean(),"len:",len(reviews_len))

min: 0 max: 351 min: 18.374603174603173 len: 56700


In [139]:
# for i in range(6,29):
print(np.count_nonzero(reviews_len > 80))

538


**Remove outliers**

In [147]:
# cleaned_rokens_arr=[x for x in tokens_arr if len(x)<81]
# print(len(cleaned_rokens_arr))

ثریا 
اینجا کد ورد امبدنیگ رو میتونی ببینی 
کلمات مختلفی هم باهاش امتحان کردم اوکی شد

# **Word embedding**

In [ ]:
!pip install gensim
# !pip install nltk

In [7]:
#main_code
import gensim
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/model.bin', binary=True)

In [ ]:
vector = model['خوب']
# see the shape of the vector (300,)
# vector.shape
vector

In [8]:
# model.most_similar('بهتر')

**calculate vacors for each word**

In [9]:
vocabulary =model.wv.vocab

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [10]:
print(len(vocabulary),type(vocabulary))

966446 <class 'dict'>


In [11]:
vectors_arr=[]
for sent in tokens_arr:#cleaned_rokens_arr:
  vec_vocab=[model[word] for word in sent if word in model.wv.vocab]# if model[word]
  vectors_arr.append(vec_vocab)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
from sys import getsizeof
getsizeof(vectors_arr)

457360

**save embedded words in csv file(too large!)>500mb**

In [ ]:
# #save vctors to array
# import csv
# with open('vectors_arr.csv', 'w', newline='') as file:
#     mywriter = csv.writer(file, delimiter=',')
#     mywriter.writerows(vectors_arr)

In [12]:
#to realease memory
tokens_arr=[]
tokens_array=[]

# **Prepare Train data for train**

## **Vectors**

**lenght of vectors**

In [13]:
vectors_len = [len(x) for x in vectors_arr]
vectors=np.array(vectors_len)
max_vector_len=max(vectors_len)

In [14]:
max_vector_len

351

In [15]:
#to realease memory
vectors_len=[]

**Add padding to short comemnts**

In [16]:
#add padding vetor 0
padding_vec=[0.0]*100
padding_vec=np.array(padding_vec)
pad_embedded_array=[]
for vector in vectors_arr:
  if len(vector)<max_vector_len:
    padding=[padding_vec]*(max_vector_len-len(vector))
    vector=vector+padding
  pad_embedded_array.append(vector)

## **Labels**

In [17]:
l=train_df["label_id"]

In [18]:
#to release memory

train_df=[]

# **Traingin LSTM**

**Import tensowerflow**

In [19]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [20]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [21]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [22]:
# from keras.layers import Embedding
from keras.layers import Input
# from keras.layers import TimeDistributed, Bidirectional
from keras.layers import LSTM
from keras.models import Model
from keras.layers.core import Dense, Dropout,Activation
# import tqdm
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
# from tensorflow.python.keras.layers import Input
from keras.models import Sequential
# from my_classes import DataGenerator
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence

In [34]:
import pathlib
import os
np.set_printoptions(precision=4)


**Create Model**

In [38]:
nb_samples = len(padding_vec)#55000
time_steps=max_vector_len#74
input_dim=100 #100
input_shape=(time_steps,input_dim)
n_epoch=6
batch_size=32

In [39]:
# create the model
# embedding_weights = np.zeros((n_symbols, vocab_dim))
lstm_model = Sequential()
# model.add(LSTM(3NumberOfLSTM, return_sequences=True,input_shape=(YourSequenceLenght, YourWord2VecLenght)))
lstm_model.add(LSTM(batch_size,return_sequences=True, input_shape=input_shape))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(40, activation='softmax'))
# lstm_model.add(Dense(20, activation='softmax'))
lstm_model.add(Dense(1, activation='softmax'))
lstm_model.add(Activation('sigmoid'))
# lstm_model.add(Activation('softmax'))
print('Compiling the Model...')
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Compiling the Model...
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 351, 32)           17024     
                                                                 
 dropout_2 (Dropout)         (None, 351, 32)           0         
                                                                 
 dense_4 (Dense)             (None, 351, 40)           1320      
                                                                 
 dense_5 (Dense)             (None, 351, 1)            41        
                                                                 
 activation_2 (Activation)   (None, 351, 1)            0         
                                                                 
Total params: 18,385
Trainable params: 18,385
Non-trainable params: 0
_________________________________________________________________


In [41]:
import keras.utils
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

AttributeError: ignored

**Train model**

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
checkpoint = ModelCheckpoint('best_model.h5', monitor='loss', mode='max', verbose=1, save_best_only=True)
x_train=pad_embedded_array
# tf.data.Dataset.from_tensors(pad_embedded_array)
y_train=l

In [ ]:
print( "Train..." )# batch_size=32
history =lstm_model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch,verbose=1,callbacks=[es, checkpoint])
# history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
# import torch

# from torch.utils.data import TensorDataset, DataLoader,Dataset
# import torchvision
# import torch.nn as nn
# class MyDataset(Dataset):
 
#   def __init__(self,dataframeX_name,dataframey_name):
#     priceX_df = dataframeX_name
#     pricey_df = dataframey_name
 
#     x = priceX_df
#     y = pricey_df.loc[:,"label_id"].values
 
#     self.x_train=torch.tensor(x,dtype=torch.float32)
#     self.y_train=torch.tensor(y,dtype=torch.float32)
 
#   def __len__(self):
#     return len(self.y_train)
   
#   def __ge‏
# ‪xed-kcap-oic‬‏

# **TEST**

**Prepare test data**

In [74]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
test_tokenized_array=[]
for i in range(0,len(dev_df)):
  comment=dev_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  test_tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


In [96]:
len(test_tokenized_array)

6300

In [77]:
(test_tokenized_array[0])

['همه',
 'ساندویچ\u200cها',
 'سرد',
 'بود',
 'متاسفانه',
 'در',
 'صورتی',
 'که',
 'فاصله',
 'تا',
 'رستوران',
 'کمتر',
 'از',
 '۵',
 'دقیقه',
 'است',
 '.']

In [79]:
# pd.DataFrame(test_tokenized_array).to_csv('testtokens_df.csv')

In [ ]:
# #save tokens to array
# import csv
# with open('test_tokens_array.csv', 'w', newline='') as file:
#     mywriter = csv.writer(file, delimiter=',')
#     mywriter.writerows(test_tokenized_arr)

In [85]:
# #read test array csv file
# test_tokens_csv=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/testtokens_df.csv",header=None)#test_tokens_array.csv"
# test_tokens_arr=test_tokens_csv.to_numpy()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [116]:
test_vectors_arr=[]
for sent in test_tokens_arr:
  vec_vocab=[model[word] for word in sent if word in vocabulary]
  test_vectors_arr.append(vec_vocab)

In [117]:
# vectors_test_len = [len(x) for x in test_vectors_arr]
# vectors_test_len=np.array(vectors_test_len)
# max_vector_test_len=max(vectors_test_len)
# max_vector_test_len

79

In [118]:
max_vector_test_len=80
#add padding vetor 0
padding_vec=[0.0]*100
padding_vec=np.array(padding_vec)
pad_vec_test_array=[]
for vector in test_vectors_arr:
  if len(vector)<max_vector_test_len:
    padding=[padding_vec]*(max_vector_test_len-len(vector))
    vector=vector+padding
  pad_vec_test_array.append(vector)

In [121]:
x_test=pad_vec_test_array
y_test=dev_df["label_id"]

In [ ]:
# evaluate the model
scores = lstm_model.evaluate(x_test, y_test, verbose=0)

Test Model